# Scraper - Cekfakta
Scraping was done on 5th September 2021 9:04 AM

In [1]:
# Standard
import pandas as pd
import os

# Crawlers
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests

# For Time Monitoring
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
import time

# Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
def readpage(url):
    req = requests.get(url, verify=False)
    page = bs(req.content, 'lxml')
    return page

In [3]:
driver = webdriver.Edge('msedgedriver.exe')

In [53]:
%%time
website = 'https://cekfakta.com/page/'
driver.get(website)

Wall time: 3.69 s


In [54]:
%%time
judul_cf = []
tanggal_cf = []
link_cf = []

# 1751

for i in tqdm_notebook(range(1000)):
    # Define Cards
    cf = bs(driver.page_source, 'lxml')
    cards = cf.find_all('li', {'class':'card'})

    # Get Judul & Link
    headers = [card.find('h1', {'class':'title'}) for card in cards]
    # Get Judul
    judul = [head.get_text().strip() for head in headers]
    judul_cf.extend(judul)
    # Get Link
    links = [head.find('a')['href'] for head in headers]
    link_cf.extend(links)
    
    # Get Tanggal
    tanggal = [card.find('span', {'class':'info'}).get_text().strip() for card in cards]
    tanggal_cf.extend(tanggal)
    
    # Next Page
    np_button = driver.find_element_by_xpath(".//div[@class='pagination-nav']/a[@rel='next']")
    np_button.click()
    time.sleep(0.25)


Wall time: 1h 11min 41s


In [55]:
len(judul_cf)

4000

In [56]:
# Make into DF
cf_df = pd.DataFrame(list(zip(judul_cf, tanggal_cf, link_cf)))
cf_df.columns = ['judul', 'tanggal', 'link']

In [57]:
# Put into CSV
cf_df.to_csv('cf_data.csv', index=False)

## Find Corona

In [ ]:
df = cf_df.copy()

In [18]:
cf = df[df['judul'].apply(lambda x: ('Covid' in x) | \
    ('Vaksin' in x) | \
    ('Pandemi' in x) | \
  ('Coronavirus' in x) | \
                    ('Masker' in x))].reset_index(drop=True)

In [21]:
cf.to_csv('cf_data_covid.csv')